# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [ ]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [ ]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [ ]:
# Your code here
salaries.head()

In [ ]:
salaries

# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [ ]:
# Your code here

# Limpiar nombres de columna y obtener empleados pagados por hora
salaries.columns = salaries.columns.str.strip()
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']
hourly_rates = hourly_workers['Hourly Rate'].dropna()

# Definir los parámetros para la prueba de hipótesis
mu_0 = 30  # Media hipotética de la población
n = len(hourly_rates)
if n == 0:
    print("No hay datos suficientes de tarifas por hora.")
    exit()

# Calcular la media y desviación estándar de la muestra
sample_mean = np.mean(hourly_rates)
sample_std = np.std(hourly_rates, ddof=1)  # ddof=1 para usar desviación estándar de muestra

# Calcular el estadístico t
t_stat = (sample_mean - mu_0) / (sample_std / np.sqrt(n))

# Calcular el valor p para prueba de dos colas
p_value = 2 * (1 - t.cdf(abs(t_stat), df=n-1))

# Imprimir los resultados
print("Estadístico t:", t_stat)
print("Valor p:", p_value)

# Interpretación
alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula: el salario por hora es significativamente diferente de $30/hora.")
else:
    print("No rechazamos la hipótesis nula: no hay evidencia suficiente para afirmar que el salario por hora es diferente de $30/hora.")


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [ ]:
# Your code here
salaries3 = salaries['Hourly Rate'].dropna()
sample_mean = np.mean(salaries3)
sample_sem = sem(salaries3)
df = len(salaries3) - 1
confidence_interval = t.interval(0.95, df, loc=sample_mean, scale=sample_sem)
print("Intervalo de confianza del 95% para el salario por hora medio:", confidence_interval)

# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [ ]:
# Your code here

# from statsmodels.stats.proportion import proportions_ztest
workers_per_hour = salaries['Hourly Rate'].notna().sum()
total_workers = len(salaries)
population_proportion = 0.25
count = workers_per_hour
nobs = total_workers
stat, p_value = proportions_ztest(count, nobs, population_proportion)
alpha = 0.05
if p_value < alpha:
    print("Rechazamos la hipótesis nula: la proporción de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("No rechazamos la hipótesis nula: no hay evidencia suficiente para decir que la proporción de trabajadores por hora es diferente del 25%.")
print(f"Estadístico z: {stat:.4f}, Valor p: {p_value:.4e}")